[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Bottle Quality Inspection Using YOLO**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## Overview

This notebook demonstrates an end-to-end computer vision pipeline for automated bottle and cap quality inspection using a YOLO-based segmentation and tracking model. The workflow covers dataset preparation, annotation, model training, inference, and real-time inspection logic on industrial video footage.


#### Real-World Applications:
Automated bottle inspection in manufacturing lines

Quality assurance for FMCG and packaging industries

Real-time defect detection (missing caps)

Industrial computer vision and smart factories

Conveyor belt monitoring and counting systems

## Annotate your Custom dataset using Labellerr

 ***1. Visit the [Labellerr](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) website and click **“Sign Up”**.*** 

 ***2. After signing in, create your workspace by entering a unique name.***

 ***3. Navigate to your workspace’s API keys page (e.g., `https://<your-workspace>.labellerr.com/workspace/api-keys`) to generate your **API Key** and **API Secret**.***

 ***4. Store the credentials securely, and then use them to initialise the SDK or API client with `api_key`, `api_secret`.*** 



## Import Libraries

This section imports all the required libraries used throughout the project for computer vision, visualization, deep learning, and structured coding.


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import torch
from typing import List, Tuple, Dict
from pathlib import Path

In [ ]:
# !git clone https://github.com/Labellerr/yolo_finetune_utils.git

## 🎞️ Random Frame Extraction from Video

Extracts a fixed number of high-quality frames from one or more videos to create an image dataset for annotation and training.

### 🔹 Purpose
- Convert raw manufacturing videos into individual image frames  
- Perform random sampling to avoid frame bias  
- Prepare data for annotation and YOLO training  


In [ ]:
from yolo_finetune_utils.frame_extractor import extract_random_frames

extract_random_frames(
    paths=['/content/sample1 (1).mp4'],
    total_images=50,
    out_dir="manufacturing_dataset_frames",
    jpg_quality=100,
    seed=42
)

## 📥 Download Annotations from Labellerr

After completing data labeling on the **Labellerr** platform, export the annotations in **COCO JSON format**.

Download the COCO JSON file from the Labellerr website and upload it into this project workspace to use it for further dataset preparation and training.

This COCO JSON file will be used in the next steps for:
- Frame–annotation alignment
- COCO → YOLO format conversion
- Model training and evaluation


In [ ]:
import os

os.rename(
    "export-#QhnW4ewqwaZwPkzCpvYS.json",
    "New_Annotation.json"
)


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.9.1+cpu
False


In [ ]:
import json
from pathlib import Path

IMG_DIR = Path("manufacturing_dataset_frames")
ANN_FILE = Path("New_Annotation.json")

with open(ANN_FILE, "r") as f:
    coco = json.load(f)

annotated_images = {img["file_name"] for img in coco["images"]}
actual_images = {p.name for p in IMG_DIR.glob("*.jpg")}

print("Matching images:", len(actual_images & annotated_images))


Matching images: 50


# COCO to YOLO Format Conversion

Converts COCO-style segmentation annotations to YOLO segmentation dataset format.  
- Requires: `annotation.json` and images in `frames_output` directory.
- Output: Generated YOLO dataset folder.
- Parameters: allows train/val split, shuffling, and verbose mode.


In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

coco_to_yolo_converter(
    json_path="New_Annotation.json",
    images_dir="manufacturing_dataset_frames",
    output_dir="yolo_dataset",
    use_split=True,
    train_ratio=0.7,
    val_ratio=0.2,
    test_ratio=0.1,
    shuffle=True,
    verbose=True
)


Conversion complete. Stats: {'train': 34, 'val': 10, 'test': 5}


{'stats': {'train': 34, 'val': 10, 'test': 5}, 'output_dir': 'yolo_dataset'}

In [ ]:
from pathlib import Path

print(Path("yolo_dataset/data.yaml").exists())
print(len(list(Path("yolo_dataset/images/train").glob("*.jpg"))))
print(len(list(Path("yolo_dataset/labels/train").glob("*.txt"))))


True
34
34


# Load and Train YOLO Segmentation Model

Loads the YOLO segmentation model and trains it using the converted YOLO dataset.
- Data: Path to YOLO-style `data.yaml`
- Parameters: epochs, image size, batch size, device, dataloader workers, experiment name.


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")

model.train(
    data="yolo_dataset/data.yaml",
    epochs=25,
    imgsz=640,
    batch=2,
    device="cpu",
    workers=2
)

Ultralytics 8.3.248  Python-3.11.9 torch-2.9.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EE3030AE10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0

## 🔍 Find Trained Model Weights

This step locates the `best.pt` file generated after YOLO training, which is used for inference.


In [ ]:
import os

for root, dirs, files in os.walk("runs"):
    if "best.pt" in files:
        print(os.path.join(root, "best.pt"))


runs\segment\train4\weights\best.pt
runs\segment\train5\weights\best.pt


## 🎥 Run Inference on Video (Memory-Safe)

This step runs YOLO inference on the input video using the trained `best.pt` model.  
Streaming mode is enabled to avoid RAM overflow while processing long videos, and results are saved automatically.

- Uses CPU inference  
- Processes frames in a streaming manner  
- Applies confidence filtering and frame skipping for stability  
- Saves the output video with detections


In [ ]:
from ultralytics import YOLO

MODEL_PATH = r"runs\segment\train5\weights\best.pt"
VIDEO_PATH = r"sample1 (2).mp4"

model = YOLO(MODEL_PATH)

for _ in model.predict(
    source=VIDEO_PATH,
    save=True,
    stream=True,      
    imgsz=416,
    device="cpu",
    workers=0,
    conf=0.6,
    vid_stride=2      
):
    pass

print("✅ Full video processed without RAM crash")



video 1/1 (frame 1/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 99.0ms
video 1/1 (frame 2/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 61.9ms
video 1/1 (frame 3/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 58.5ms
video 1/1 (frame 4/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 62.7ms
video 1/1 (frame 5/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 3 bottles, 63.0ms
video 1/1 (frame 6/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 59.6ms
video 1/1 (frame 7/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 58.4ms
video 1/1 (frame 8/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 61.0ms
video 1/1 (frame 9/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 59.5ms
video 1/1 (frame 10/345) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 55.1ms
video 1/1 (frame 11/345) d:\Desktop\Projects\sam

## 🧪 Bottle–Cap Quality Inspection (ROI + Tracking)

This script performs **end-to-end quality inspection** on a conveyor-style video using a trained **YOLOv8 segmentation model**.

### What this code does
- Loads a trained YOLO model (`best.pt`)
- Tracks **bottles and caps** across video frames
- Defines a **Rectangular ROI (Region of Interest)** for inspection
- Applies a **transparent overlay** to visually highlight the ROI
- Uses an **inspection line** to count objects moving **left → right**
- Associates **caps with bottles** using bounding-box alignment logic
- Classifies each bottle as **PASS / FAIL**
- Maintains **live counters**:
  - Total Bottles
  - Total Caps
  - Total Defective Bottles
- Writes a fully annotated **output inspection video**

### Key Features
- ✅ Object tracking with persistent IDs  
- ✅ Memory-safe streaming inference  
- ✅ ROI-based filtering (reduces false detections)  
- ✅ Visual inspection aids (overlay, counters, inspection line)  
- ✅ Final output saved as an annotated MP4 video  

### Output
- Annotated video with:
  - ROI overlay
  - Bottle & cap detections
  - PASS / FAIL labels
  - Live counters
- Final inspection statistics printed to terminal

📹 **Output file:** `output_inspection_LR.mp4`


In [ ]:
import cv2
from ultralytics import YOLO


MODEL_PATH = r"runs\segment\train5\weights\best.pt"
VIDEO_PATH = r"sample1 (2).mp4"
OUTPUT_VIDEO = r"output_inspection_LR.mp4"


BOTTLE_CLASS = 0
CAP_CLASS = 1

BOTTLE_CONF = 0.7
CAP_CONF = 0.7

# ROI (normalized)
ROI = {
    "x1": 0.6,
    "y1": 0.2,
    "x2": 0.9,
    "y2": 1.0
}

INSPECTION_X_RATIO = 0.75  

# ROI visuals
ROI_COLOR = (255, 0, 0)
ROI_BORDER_THICKNESS = 8
ROI_ALPHA = 0.25


def inside_roi(cx, cy, W, H, roi):
    return (
        roi["x1"] * W <= cx <= roi["x2"] * W and
        roi["y1"] * H <= cy <= roi["y2"] * H
    )


bottle_ids_crossed = set()
cap_ids_crossed = set()


bottle_cap_status = {}

bottle_count = 0
cap_count = 0
defect_count = 0

# =========================
# MODEL
# =========================
model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(VIDEO_PATH)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS = cap.get(cv2.CAP_PROP_FPS)
cap.release()

writer = cv2.VideoWriter(
    OUTPUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (W, H)
)

print("🚀 Running LEFT → RIGHT bottle inspection...")

# =========================
# TRACKING LOOP
# =========================
for r in model.track(
    source=VIDEO_PATH,
    stream=True,
    persist=True,
    conf=0.25,
    imgsz=416,
    device="cpu",
    workers=0
):

    frame = r.orig_img.copy()
    overlay = frame.copy()
    H, W = frame.shape[:2]

    # ROI coordinates
    roi_x1 = int(ROI["x1"] * W)
    roi_y1 = int(ROI["y1"] * H)
    roi_x2 = int(ROI["x2"] * W)
    roi_y2 = int(ROI["y2"] * H)

    inspection_x = int(INSPECTION_X_RATIO * W)

    # ================= ROI TRANSPARENT OVERLAY =================
    cv2.rectangle(
        overlay,
        (roi_x1, roi_y1),
        (roi_x2, roi_y2),
        ROI_COLOR,
        -1
    )

    cv2.addWeighted(
        overlay,
        ROI_ALPHA,
        frame,
        1 - ROI_ALPHA,
        0,
        frame
    )

    # ROI border
    cv2.rectangle(
        frame,
        (roi_x1, roi_y1),
        (roi_x2, roi_y2),
        ROI_COLOR,
        ROI_BORDER_THICKNESS
    )

    cv2.putText(
        frame,
        "ROI",
        (roi_x1, roi_y1 - 15),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,
        ROI_COLOR,
        3
    )

    # Inspection line
    cv2.line(frame, (inspection_x, 0), (inspection_x, H), (0, 255, 255), 4)

    if r.boxes is None or r.boxes.id is None:
        writer.write(frame)
        continue

    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clss = r.boxes.cls.cpu().numpy()
    ids = r.boxes.id.cpu().numpy()

    # ================= CAP → BOTTLE ASSOCIATION =================
    for (x1, y1, x2, y2), conf, cls, tid in zip(boxes, confs, clss, ids):

        if cls != CAP_CLASS or conf < CAP_CONF:
            continue

        cap_cx = int((x1 + x2) / 2)
        cap_cy = int((y1 + y2) / 2)

        if not inside_roi(cap_cx, cap_cy, W, H, ROI):
            continue

        # Draw cap
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3)
        cv2.putText(frame, f" {conf:.2f}", (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Count cap crossing
        if cap_cx >= inspection_x and tid not in cap_ids_crossed:
            cap_ids_crossed.add(tid)
            cap_count += 1

        # Match cap to bottle by X containment
        for (bx1, by1, bx2, by2), b_conf, b_cls, b_tid in zip(boxes, confs, clss, ids):
            if b_cls != BOTTLE_CLASS or b_conf < BOTTLE_CONF:
                continue

            if bx1 <= cap_cx <= bx2:
                bottle_cap_status[b_tid] = True

    # ================= BOTTLES =================
    for (x1, y1, x2, y2), conf, cls, tid in zip(boxes, confs, clss, ids):

        if cls != BOTTLE_CLASS or conf < BOTTLE_CONF:
            continue

        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        if not inside_roi(cx, cy, W, H, ROI):
            continue

        # Initialize if new bottle
        if tid not in bottle_cap_status:
            bottle_cap_status[tid] = False

        status = "PASS" if bottle_cap_status[tid] else "FAIL"
        color = (0, 255, 0) if status == "PASS" else (0, 0, 255)

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 3)
        cv2.putText(frame, f" {conf:.2f}", (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # LEFT → RIGHT crossing
        if cx >= inspection_x and tid not in bottle_ids_crossed:
            bottle_ids_crossed.add(tid)
            bottle_count += 1
            if not bottle_cap_status[tid]:
                defect_count += 1

    # ================= COUNTER PANEL (4× SIZE) =================
    cv2.rectangle(frame, (20, 20), (620, 360), (0, 0, 0), -1)

    cv2.putText(frame, f"Bottles: {bottle_count}", (40, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 1.6, (255, 255, 255), 3)
    cv2.putText(frame, f"Caps: {cap_count}", (40, 220),
                cv2.FONT_HERSHEY_SIMPLEX, 1.6, (255, 255, 255), 3)
    cv2.putText(frame, f"Defects: {defect_count}", (40, 320),
                cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 0, 255), 3)

    writer.write(frame)

writer.release()

print("✅ Inspection complete")
print(f"Total bottles: {bottle_count}")
print(f"Total caps: {cap_count}")
print(f"Defects: {defect_count}")
print(f"📹 Output saved as: {OUTPUT_VIDEO}")


🚀 Running LEFT → RIGHT bottle inspection...

video 1/1 (frame 1/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 405.3ms
video 1/1 (frame 2/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 482.5ms
video 1/1 (frame 3/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 194.9ms
video 1/1 (frame 4/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 178.3ms
video 1/1 (frame 5/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 138.1ms
video 1/1 (frame 6/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 133.7ms
video 1/1 (frame 7/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 138.0ms
video 1/1 (frame 8/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 177.6ms
video 1/1 (frame 9/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 111.8ms
video 1/1 (frame 10/690) d:\Desktop\Projects\sample1 (2).mp4: 256x416 2 caps, 4 bottles, 16

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
